# Junk testing notebook not the final model

In [1]:
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
x = np.random.randn(13, 5)
print(x)

[[-0.64149669  0.69884429 -0.09923077  1.44719497 -0.39937688]
 [-1.57425213 -0.32263962  0.08185847  0.39051642  0.91326861]
 [ 0.37046333  0.64824388  0.3414695  -0.03425428  0.22262136]
 [ 0.34248392  1.63617858  1.25776522 -0.3087631   0.78926198]
 [-0.77981961 -0.07972376  1.07504756  0.96321429  0.33597874]
 [-0.53815735  2.39853366  0.65107899 -0.76856517 -0.53599128]
 [-1.57065663  0.69091467 -1.49945899  0.49739481  0.33475268]
 [ 0.36886721  0.93643886 -0.11605022 -0.46051834  0.10699717]
 [ 0.06518237 -0.57575168  0.07757798 -1.87890303 -0.08600087]
 [ 0.52027297 -0.31057518 -0.61645971 -0.06247435  0.3512117 ]
 [-0.68904986 -0.78242203 -0.90296344  0.28954674 -0.49460374]
 [ 1.0085212  -0.10302162  0.40371888  0.43977026 -0.28065346]
 [-0.69721952 -0.15639741  1.22729461  0.54655177 -0.21798084]]


In [5]:
emb = tf.constant(x)
print(emb)

Tensor("Const:0", shape=(13, 5), dtype=float64)


In [6]:
q = [1]
q_pl = tf.placeholder(tf.int32, [None], name = 'query')

In [7]:
lookup = tf.nn.embedding_lookup(emb, q_pl)

In [8]:
sess = tf.Session()

In [9]:
sess.run(lookup, feed_dict = {q_pl: q})

array([[-1.57425213, -0.32263962,  0.08185847,  0.39051642,  0.91326861]])

In [13]:
sess.run(tf.sigmoid([12., 0.]))

array([0.9999938, 0.5      ], dtype=float32)

In [15]:
batch_ = np.random.randn(10, 4)
print(batch_)
print(batch_[:, 1:])

[[-0.46150963  1.65220658 -1.22201059 -0.26363355]
 [ 0.22971067  1.33664605  0.11253774 -0.07970056]
 [ 1.92576167  0.73469331 -0.75642091  1.51890203]
 [ 1.20769575 -0.08556157 -0.06427001  0.36002096]
 [-1.68428291  0.24842158  1.15771564  0.52042367]
 [-1.14990415  0.03939468 -0.39121161 -0.47556889]
 [ 1.37515466  0.0112242  -0.40831684 -1.60524304]
 [ 1.60873132  1.29707674 -0.90152081 -0.57840124]
 [-0.51112579 -0.976894   -1.90939731  0.53125335]
 [-0.94501113  1.11559169 -0.61929821 -0.54978269]]
[[ 1.65220658 -1.22201059 -0.26363355]
 [ 1.33664605  0.11253774 -0.07970056]
 [ 0.73469331 -0.75642091  1.51890203]
 [-0.08556157 -0.06427001  0.36002096]
 [ 0.24842158  1.15771564  0.52042367]
 [ 0.03939468 -0.39121161 -0.47556889]
 [ 0.0112242  -0.40831684 -1.60524304]
 [ 1.29707674 -0.90152081 -0.57840124]
 [-0.976894   -1.90939731  0.53125335]
 [ 1.11559169 -0.61929821 -0.54978269]]


In [37]:
sequences = []
batch_size = 10
for i in range(batch_size):
    seq = np.random.randn(np.random.randint(1, 6)).astype(np.float32)
    sequences.append(seq)

sequences = np.array(sequences)
print(sequences)

[array([-1.5097468 ,  0.97187704, -1.1467037 ,  1.628563  ], dtype=float32)
 array([ 0.08486259, -1.4896799 ,  1.6080999 ], dtype=float32)
 array([-0.21923354, -1.9496495 , -2.5362346 , -0.98984426, -0.07178294],
      dtype=float32)
 array([-1.360168  , -0.18784778, -0.16101228, -0.8402991 ], dtype=float32)
 array([-0.86216503], dtype=float32)
 array([-1.0983992 ,  0.6750412 , -0.39504507], dtype=float32)
 array([ 0.63389575, -0.40742725,  0.6196666 ,  1.7046864 , -0.49599558],
      dtype=float32)
 array([-1.1983697 ,  0.3309764 ,  0.6203675 , -0.98709786,  0.61874163],
      dtype=float32)
 array([2.010768], dtype=float32)
 array([-0.39132604, -0.8241489 , -1.0844672 ,  0.8207169 ], dtype=float32)]


In [38]:
sequences.shape

(10,)

In [39]:
pad_id = 99.

In [40]:
seqlens = [i.shape[0] for i in sequences]
# print(seqlens)
for s in sequences:
    seq = np.append(arr = s, values = np.ones(5 - len(s),dtype = np.int32) * pad_id)
    print(seq)

[-1.50974679  0.97187704 -1.14670372  1.62856305 99.        ]
[ 8.48625898e-02 -1.48967993e+00  1.60809994e+00  9.90000000e+01
  9.90000000e+01]
[-0.21923354 -1.94964945 -2.53623462 -0.98984426 -0.07178294]
[-1.36016798 -0.18784778 -0.16101228 -0.84029913 99.        ]
[-0.86216503 99.         99.         99.         99.        ]
[-1.09839916  0.6750412  -0.39504507 99.         99.        ]
[ 0.63389575 -0.40742725  0.61966658  1.7046864  -0.49599558]
[-1.19836974  0.3309764   0.62036753 -0.98709786  0.61874163]
[ 2.01076794 99.         99.         99.         99.        ]
[-0.39132604 -0.82414889 -1.08446717  0.82071692 99.        ]


In [ ]:
inp = tf.placeholder(tf.int32, [None])

In [41]:
s

array([-0.39132604, -0.8241489 , -1.0844672 ,  0.8207169 ], dtype=float32)

In [42]:
x_l = [i*i for i in range(3)]
x_t = (i*i for i in range(3))

In [43]:
for i in x_l:
    print(i)
print()

for i in x_t:
    print(i)

0
1
4

0
1
4


In [45]:
for i in x_l:
    print(i)
print('here')

for i in x_t:
    print(i)

0
1
4
here
